## Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [ ]:
# Setup - Run this cell first
import polars as pl

# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values='NA')
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values='NA')
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', schema_overrides={'precip': pl.Float64, 'visib': pl.Float64}, null_values='NA')

flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-3104250466.py:15: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [ ]:
# Write your SQL query here
result1 = ctx.execute("""
SELECT DISTINCT carrier
    FROM airlines
    ORDER BY carrier
""")


print(result1)

shape: (16, 1)
┌─────────┐
│ carrier │
│ ---     │
│ str     │
╞═════════╡
│ 9E      │
│ AA      │
│ AS      │
│ B6      │
│ DL      │
│ …       │
│ UA      │
│ US      │
│ VX      │
│ WN      │
│ YV      │
└─────────┘


### 1.2 Find the top 10 destinations by number of flights

In [13]:
# Write your SQL query here
result2 = ctx.execute("""
SELECT dest, COUNT(*) AS flights
    FROM flights
    GROUP BY dest
    ORDER BY flights DESC
    LIMIT 10
""")

# print(result)
print(result2)

shape: (10, 2)
┌──────┬─────────┐
│ dest ┆ flights │
│ ---  ┆ ---     │
│ str  ┆ u32     │
╞══════╪═════════╡
│ ORD  ┆ 17283   │
│ ATL  ┆ 17215   │
│ LAX  ┆ 16174   │
│ BOS  ┆ 15508   │
│ MCO  ┆ 14082   │
│ CLT  ┆ 14064   │
│ SFO  ┆ 13331   │
│ FLL  ┆ 12055   │
│ MIA  ┆ 11728   │
│ DCA  ┆ 9705    │
└──────┴─────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [14]:
# Write your SQL query here
result3 = ctx.execute("""
SELECT *
    FROM flights
    WHERE dep_delay > 120
""")

# print(result)

print(result3)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [15]:
# Write your SQL query here
result4 = ctx.execute("""
SELECT origin, AVG(dep_delay) AS avg_dep_delay
    FROM flights
    GROUP BY origin
    ORDER BY avg_dep_delay DESC
""")

# print(result)

print(result4)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ EWR    ┆ 15.107954     │
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [16]:
# First, let's check what columns are available
result5 = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")
# print(result)

print(result5)

# Now write your query to find busiest month
result6 = ctx.execute("""
SELECT month, COUNT(*) AS flights
    FROM flights
    GROUP BY month
    ORDER BY flights DESC
    LIMIT 1
""")

# print(result)

print(result6)

shape: (5, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 1400     ┆ 5    ┆ 15     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 1416     ┆ 5    ┆ 29     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 1089     ┆ 5    ┆ 40     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 544      ┆ … ┆ 1576     ┆ 5    ┆ 45     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 554      ┆ … ┆ 762      ┆ 6    ┆ 0      ┆ 2013-01-01 11:00:00 UTC │
└──────┴───────┴─────┴──────────┴───┴──────────┴──────┴────────┴───────────

### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [17]:
# Write your SQL query here
result7 = ctx.execute("""
SELECT
        carrier,
        AVG(CASE WHEN dep_delay <= 15 THEN 1.0 ELSE 0.0 END) AS on_time_rate
    FROM flights
    GROUP BY carrier
    ORDER BY on_time_rate DESC
""")

# print(result)

print(result7)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ HA      ┆ 0.929825     │
│ AS      ┆ 0.865546     │
│ US      ┆ 0.849873     │
│ DL      ┆ 0.830742     │
│ AA      ┆ 0.824376     │
│ …       ┆ …            │
│ FL      ┆ 0.716871     │
│ F9      ┆ 0.715328     │
│ 9E      ┆ 0.701246     │
│ EV      ┆ 0.659221     │
│ YV      ┆ 0.647255     │
└─────────┴──────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [18]:
# Write your SQL query here
result8 = ctx.execute("""
SELECT
        f.carrier,
        a.name AS airline_name,
        f.flight,
        f.origin,
        f.dest
    FROM flights AS f
    JOIN airlines AS a
      ON f.carrier = a.carrier
    ORDER BY f.time_hour
    LIMIT 20
""")

# print(result)

print(result8)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ UA      ┆ United Air Lines Inc.  ┆ 1696   ┆ EWR    ┆ ORD  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ AA      ┆ American Airlines Inc. ┆ 707    ┆ LGA    ┆ DFW  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [19]:
# Write your SQL query here
result9 = ctx.execute("""
SELECT
        f.carrier,
        AVG(2013 - p.year) AS avg_aircraft_age
    FROM flights AS f
    JOIN planes  AS p
      ON f.tailnum = p.tailnum
    WHERE p.year IS NOT NULL
    GROUP BY f.carrier
    ORDER BY avg_aircraft_age DESC
""")

# print(result)

print(result9)

shape: (16, 2)
┌─────────┬──────────────────┐
│ carrier ┆ avg_aircraft_age │
│ ---     ┆ ---              │
│ str     ┆ f64              │
╞═════════╪══════════════════╡
│ MQ      ┆ 35.319           │
│ AA      ┆ 25.869426        │
│ DL      ┆ 16.372169        │
│ UA      ┆ 13.207691        │
│ FL      ┆ 11.385829        │
│ …       ┆ …                │
│ B6      ┆ 6.686702         │
│ F9      ┆ 4.87874          │
│ VX      ┆ 4.473643         │
│ AS      ┆ 3.33662          │
│ HA      ┆ 1.548387         │
└─────────┴──────────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [20]:
# First, explore the weather table structure
result10 = ctx.execute("""
    SELECT *
    FROM weather
    LIMIT 5
""")
# print(result)

print(result10)

# Now write your join query
result11 = ctx.execute("""
SELECT
        f.*,
        w.wind_speed,
        w.precip
    FROM flights AS f
    JOIN weather AS w
      ON f.origin = w.origin
     AND f.time_hour = w.time_hour
    WHERE f.dep_delay > 30
      AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

# print(result)

print(result11)

shape: (5, 15)
┌────────┬──────┬───────┬─────┬───┬────────┬──────────┬───────┬─────────────────────────┐
│ origin ┆ year ┆ month ┆ day ┆ … ┆ precip ┆ pressure ┆ visib ┆ time_hour               │
│ ---    ┆ ---  ┆ ---   ┆ --- ┆   ┆ ---    ┆ ---      ┆ ---   ┆ ---                     │
│ str    ┆ i64  ┆ i64   ┆ i64 ┆   ┆ f64    ┆ f64      ┆ f64   ┆ datetime[μs, UTC]       │
╞════════╪══════╪═══════╪═════╪═══╪════════╪══════════╪═══════╪═════════════════════════╡
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.0   ┆ 10.0  ┆ 2013-01-01 06:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.3   ┆ 10.0  ┆ 2013-01-01 07:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.5   ┆ 10.0  ┆ 2013-01-01 08:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1012.2   ┆ 10.0  ┆ 2013-01-01 09:00:00 UTC │
│ EWR    ┆ 2013 ┆ 1     ┆ 1   ┆ … ┆ 0.0    ┆ 1011.9   ┆ 10.0  ┆ 2013-01-01 10:00:00 UTC │
└────────┴──────┴───────┴─────┴───┴────────┴──────────┴───────┴──────────────────────

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [21]:
# Write your SQL query here
result12 = ctx.execute("""
SELECT
        p.manufacturer,
        p.model,
        COUNT(*) AS flights
    FROM flights AS f
    JOIN planes  AS p
      ON f.tailnum = p.tailnum
    GROUP BY p.manufacturer, p.model
    ORDER BY flights DESC
    LIMIT 10
""")

# print(result)

print(result12)

shape: (10, 3)
┌───────────────────────────────┬─────────────────┬─────────┐
│ manufacturer                  ┆ model           ┆ flights │
│ ---                           ┆ ---             ┆ ---     │
│ str                           ┆ str             ┆ u32     │
╞═══════════════════════════════╪═════════════════╪═════════╡
│ AIRBUS                        ┆ A320-232        ┆ 31278   │
│ EMBRAER                       ┆ EMB-145LR       ┆ 28027   │
│ EMBRAER                       ┆ ERJ 190-100 IGW ┆ 23716   │
│ AIRBUS INDUSTRIE              ┆ A320-232        ┆ 14553   │
│ EMBRAER                       ┆ EMB-145XR       ┆ 14051   │
│ BOEING                        ┆ 737-824         ┆ 13809   │
│ BOMBARDIER INC                ┆ CL-600-2D24     ┆ 11807   │
│ BOEING                        ┆ 737-7H4         ┆ 10389   │
│ BOEING                        ┆ 757-222         ┆ 9150    │
│ MCDONNELL DOUGLAS AIRCRAFT CO ┆ MD-88           ┆ 8932    │
└───────────────────────────────┴─────────────────┴────

### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [29]:
flights_with_airport_names = flights.join(
    airports,
    left_on="origin",
    right_on="faa",
    how="left"
).rename({"name": "origin_name"}).join(
    airports,
    left_on="dest",
    right_on="faa",
    how="left"
).rename({"name": "dest_name"})

ctx_joined = pl.SQLContext(
    flights_with_airport_names=flights_with_airport_names,
    eager_execution=True
)

/tmp/ipython-input-3138097619.py:13: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx_joined = pl.SQLContext(


In [30]:
# Write your SQL query here

result13 = ctx_joined.execute("""
SELECT
        origin,
        origin_name,
        dest,
        dest_name,
        COUNT(*)                                         AS total_flights,
        AVG(dep_delay)                                 AS avg_dep_delay,
        AVG(CASE WHEN dep_delay > 30 THEN 1.0 ELSE 0.0 END) AS pct_delayed_over_30
    FROM flights_with_airport_names
    GROUP BY origin, origin_name, dest, dest_name
    ORDER BY total_flights DESC
    LIMIT 10
""")

# print(result)

print(result13)

shape: (10, 7)
┌────────┬────────────────┬──────┬────────────────┬───────────────┬───────────────┬────────────────┐
│ origin ┆ origin_name    ┆ dest ┆ dest_name      ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_ov │
│ ---    ┆ ---            ┆ ---  ┆ ---            ┆ ---           ┆ ---           ┆ er_30          │
│ str    ┆ str            ┆ str  ┆ str            ┆ u32           ┆ f64           ┆ ---            │
│        ┆                ┆      ┆                ┆               ┆               ┆ f64            │
╞════════╪════════════════╪══════╪════════════════╪═══════════════╪═══════════════╪════════════════╡
│ JFK    ┆ John F Kennedy ┆ LAX  ┆ Los Angeles    ┆ 11262         ┆ 8.522508      ┆ 0.098295       │
│        ┆ Intl           ┆      ┆ Intl           ┆               ┆               ┆                │
│ LGA    ┆ La Guardia     ┆ ATL  ┆ Hartsfield     ┆ 10263         ┆ 11.448621     ┆ 0.122479       │
│        ┆                ┆      ┆ Jackson        ┆               ┆         

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [31]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here

on_time_by_carrier = (
    flights
    .select([
        pl.col("carrier"),
        (pl.col("dep_delay") <= 15).cast(pl.Float64).alias("on_time_flag")
    ])
    .group_by("carrier")
    .agg(pl.col("on_time_flag").mean().alias("on_time_rate"))
    .sort("on_time_rate", descending=True)
)

SQL Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘

Polars Result:
shape: (3, 2)
┌────────┬───────────┐
│ origin ┆ avg_delay │
│ ---    ┆ ---       │
│ str    ┆ f64       │
╞════════╪═══════════╡
│ EWR    ┆ 15.107954 │
│ JFK    ┆ 12.112159 │
│ LGA    ┆ 10.346876 │
└────────┴───────────┘
